In [1]:
# Small LSTM Network to Generate Text for Alice in Wonderland
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

C:\Users\lobof\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# load ascii text and covert to lowercase
filename = "moes_tavern_lines.txt"
source_text = open(filename).read()
source_text = source_text.lower()

In [4]:
# create mapping of unique chars to integers
chars = sorted(list(set(source_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [5]:
# summarize the loaded data
n_chars = len(source_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  305284
Total Vocab:  67


In [7]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
input_data = []
target_data = []
for i in range(0, n_chars - seq_length, 1):
    seq_input = source_text[i:i + seq_length]
    seq_target = source_text[i + seq_length]
    input_data.append([char_to_int[char] for char in seq_input])
    target_data.append(char_to_int[seq_target])
n_patterns = len(input_data)
print ("Total Patterns: ", n_patterns)

Total Patterns:  305184


In [9]:
# reshape data_input to be [samples, time steps, features]
num_reshape = numpy.reshape(input_data, (n_patterns, seq_length, 1))
# normalize
normalize_input = num_reshape/ float(n_vocab)
# one hot encode the output variable
one_hot = np_utils.to_categorical(target_data)

In [10]:
print(normalize_input)
print(one_hot)

[[[0.41791045]
  [0.82089552]
  [0.52238806]
  ...
  [0.65671642]
  [0.74626866]
  [0.67164179]]

 [[0.82089552]
  [0.52238806]
  [0.46268657]
  ...
  [0.74626866]
  [0.67164179]
  [0.01492537]]

 [[0.52238806]
  [0.46268657]
  [0.71641791]
  ...
  [0.67164179]
  [0.01492537]
  [0.68656716]]

 ...

 [[0.73134328]
  [0.01492537]
  [0.46268657]
  ...
  [0.82089552]
  [0.73134328]
  [0.52238806]]

 [[0.01492537]
  [0.46268657]
  [0.62686567]
  ...
  [0.73134328]
  [0.52238806]
  [0.62686567]]

 [[0.46268657]
  [0.62686567]
  [0.62686567]
  ...
  [0.52238806]
  [0.62686567]
  [0.53731343]]]
[[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [15]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(normalize_input.shape[1], normalize_input.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(one_hot.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [16]:
# define the checkpoint
filepath="weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [18]:
# fit the model
history = model.fit(normalize_input, one_hot, epochs=1, batch_size=64, callbacks=callbacks_list)

Epoch 1/1
305184/305184 [==============================] - 1711s 6ms/step - loss: 2.6616 - acc: 0.2652

Epoch 00001: loss improved from inf to 2.66162, saving model to weights.hdf5


In [33]:
# Load LSTM network and generate text
import sys
# pick a random seed
start = numpy.random.randint(0, len(input_data)-1)
pattern = input_data[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# Generate random characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
prediction = model.predict(x, verbose=0)
index = numpy.argmax(prediction)
result = int_to_char[index]
seq_in = [int_to_char[value] for value in pattern]
sys.stdout.write(result)
pattern.append(index)
pattern = pattern[1:len(pattern)]
print ('\n',"Script Genereated.")

Seed:
" pson: witty. (chuckles)
lenny_leonard: great meeting though.
bar_rag: yes, that's right, everyone la "
 
 Script Genereated.
